## 06_workflow.ipynb

This is the final tutorial and here we are going to explore the automated version of what we saw so far, i.e. the workflow

Table of contents:
1. overview
2. simple example
3. full example
4. loading from disk

### 01 Overview

The workflow is essnetially a simple wrapper which execute the main use case of simulating using HFSS.
The pseudo-code the workflow implements goes as follow:

create folders to save results

for parameters in set of parameters:
    save the parameters
    build the design

    for simulation in given set of simulations:
        simulation.simulate
        save simulation results

aggregate all results given the grouping configuration from the user 
save the aggregations

every step i mentioned here was already convered in the previous tutorials

### 02 simple example

The workflow is controlled using a workflow configuration which is essnetially just a composition of all the different instances we talked about.
for example:



In [ ]:
from pysubmit import WorkflowConfig, SessionParameters, EigenmodeAnalysis

config = WorkflowConfig(
    # the session parameters are related to opening and closing the file itself
    # for example: non_graphical, or version of hfss. 
    session_parameters=SessionParameters(file_path='simple_design_readout.aedt'),
    # simulations is a dict of strings to simulation instances as we saw in the first tutorial
    simulations={'classical': EigenmodeAnalysis(setup_name='Setup1', design_name='my_design')}
)

The above configuration essentially just opens the simple_design_readout.aedt and execute the Eigenmode simulation without sweeping on anything.
The saved file would be under results/iterations/iteration0/classical.json the 'classical' name for the file comes from the key that match for the EigenmodeAnalysis.
To run the configuration one can import execute_workflow as shown below:

In [2]:
from pysubmit import execute_workflow

execute_workflow(config)

PyAEDT INFO: Parsing C:\Users\rosengrp\Documents\GitHub\pysubmit\tutorial\simple_design_readout.aedt.
PyAEDT INFO: Python version 3.11.3 (tags/v3.11.3:f3909b8, Apr  4 2023, 23:49:59) [MSC v.1934 64 bit (AMD64)].
PyAEDT INFO: File C:\Users\rosengrp\Documents\GitHub\pysubmit\tutorial\simple_design_readout.aedt correctly loaded. Elapsed time: 0m 0sec
PyAEDT INFO: PyAEDT version 0.15.1.
PyAEDT INFO: Initializing new Desktop session.
PyAEDT INFO: Log on console is enabled.
PyAEDT INFO: Log on file C:\Users\rosengrp\AppData\Local\Temp\pyaedt_rosengrp_81111850-ec97-46cf-843d-9a80045b9b57.log is enabled.
PyAEDT INFO: Log on AEDT is enabled.
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT INFO: Launching PyAEDT with gRPC plugin.
PyAEDT INFO: New AEDT session is starting on gRPC port 62645.
PyAEDT INFO: Electronics Desktop started on gRPC port: 62645 after 9.972299098968506 seconds.
PyAEDT INFO: AEDT installation Path C:\Program Files\AnsysEM\v242\Win64
PyAEDT IN

## 03 full example

In this example we are going to add sweeping over the chup_length and a script builder to change it.
Here is the full configuration:

In [1]:
from pysubmit import WorkflowConfig, SessionParameters, EigenmodeAnalysis, DataHandler
from pysubmit.workflow.builder import ScriptBuilder
from pysubmit.workflow.sweep import ZipSweep
from pysubmit import execute_workflow


config = WorkflowConfig(
    session_parameters=SessionParameters(file_path='simple_design_readout.aedt'),
    simulations={'classical': EigenmodeAnalysis(setup_name='Setup1', design_name='my_design')},

    # the builder sweep is a list of sweep instances, either ProductSweep or ZipSweep.
    # the reason it is a list is because it allows for chaining the sweepers. 
    # for example you can product sweep over two parameters and for every combination of these two do another sweep 
    # on other parameters.
    builder_sweep=[ZipSweep(parameters={'value': ['10mm', '12mm', '14mm', '16mm']})],

    # this is just a builder instance, similar to the builder tutorial
    builder=ScriptBuilder(path='script_builder_function.py'),

    # data handler is also initiated here. the workflow save the build parameters using an identifier
    # called build_parameters. so if you want to aggregate the build parameters into your simulation results 
    # this is how you should do it
    data_handler=DataHandler(grouping_config={'my_aggregation': ('build_parameters', 'classical')})
)

execute_workflow(config)

PyAEDT INFO: Parsing C:\Users\rosengrp\Documents\GitHub\pysubmit\tutorial\simple_design_readout.aedt.
PyAEDT INFO: Python version 3.11.3 (tags/v3.11.3:f3909b8, Apr  4 2023, 23:49:59) [MSC v.1934 64 bit (AMD64)].
PyAEDT INFO: File C:\Users\rosengrp\Documents\GitHub\pysubmit\tutorial\simple_design_readout.aedt correctly loaded. Elapsed time: 0m 0sec
PyAEDT INFO: PyAEDT version 0.15.1.
PyAEDT INFO: Initializing new Desktop session.
PyAEDT INFO: Log on console is enabled.
PyAEDT INFO: Log on file C:\Users\rosengrp\AppData\Local\Temp\pyaedt_rosengrp_09b3b969-e80b-4959-81c8-62fbcdc7419c.log is enabled.
PyAEDT INFO: Log on AEDT is enabled.
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT INFO: Launching PyAEDT with gRPC plugin.
PyAEDT INFO: New AEDT session is starting on gRPC port 49469.
PyAEDT INFO: Electronics Desktop started on gRPC port: 49469 after 10.209462404251099 seconds.
PyAEDT INFO: AEDT installation Path C:\Program Files\AnsysEM\v242\Win64
PyAEDT I

Note that the build parameters are taken from the sweeper only if the builder itself doesn't return a dict instance.
Meaning that if i change the script builder file to return nothing (return None) then the column in the aggregation would be 'value'.
However if it does return a dict than the build_parameters are chosen from it. in our case the function does return {'chip_base_length': some_value} therefore
the aggregation would have a column called 'chip_base_length'

### 04 Loading and Saving

The workflow configuration can be loaded from a